In [ ]:
import os
import pandas
from google.colab import drive
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
%pip install deepctr
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat,get_feature_names

In [ ]:
data = pd.read_csv("movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']
data.head()


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119
1,3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005
2,517,105,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,F,25,14,55408
3,785,2115,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,18,19,29307
4,5848,909,5,957782527,"Apartment, The (1960)",Comedy|Drama,M,50,20,20009


In [ ]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [ ]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118
2,12,6,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,0,2,13,99
3,21,112,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,1,1,18,55
4,187,45,5,957782527,"Apartment, The (1960)",Comedy|Drama,1,5,19,41


In [ ]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)


In [15]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [21]:
# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=11, verbose=True, validation_split=0.2, )


Epoch 1/11


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 283ms/step - loss: 13.7655 - mse: 13.7655 - val_loss: 15.5737 - val_mse: 15.5737
Epoch 2/11
1/1 [==============================] - 0s 19ms/step - loss: 13.6346 - mse: 13.6346 - val_loss: 15.4515 - val_mse: 15.4515
Epoch 3/11
1/1 [==============================] - 0s 18ms/step - loss: 13.4929 - mse: 13.4929 - val_loss: 15.3208 - val_mse: 15.3208
Epoch 4/11
1/1 [==============================] - 0s 20ms/step - loss: 13.3424 - mse: 13.3424 - val_loss: 15.1830 - val_mse: 15.1830
Epoch 5/11
1/1 [==============================] - 0s 18ms/step - loss: 13.1834 - mse: 13.1834 - val_loss: 15.0372 - val_mse: 15.0372
Epoch 6/11
1/1 [==============================] - 0s 18ms/step - loss: 13.0157 - mse: 13.0157 - val_loss: 14.8828 - val_mse: 14.8828
Epoch 7/11
1/1 [==============================] - 0s 20ms/step - loss: 12.8388 - mse: 12.8388 - val_loss: 14.7192 - val_mse: 14.7192
Epoch 8/11
1/1 [==============================] - 0s 20ms/step - loss: 12.6518 

In [22]:
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
#print("test MSE", mse)
print("test RMSE", rmse)

test RMSE 3.522825570476063
